In [46]:
import numpy as np 
import pickle 

E = np.load('embeddings.npy')

with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [47]:
pos_tweets = []
with open('train_pos.txt') as f:
    for line in f:
        pos_tweets.append(line.split())
        
neg_tweets = []
with open('train_neg.txt') as f:
    for line in f:
        neg_tweets.append(line.split())

In [48]:
def tweet_to_coords(tweet):
    components = []
    for word in tweet: 
        if word in vocab: 
            components.append(vocab[word])
    if (len(components) > 0):
        return E[components].mean(axis = 0)
    else: 
        return np.zeros(20) # TODO: find a representation for tweets that have only rare (<5 occurences) words
        


In [57]:
pos_tweets_coords = np.array([list(map(tweet_to_coords, pos_tweets))]).squeeze()
neg_tweets_coords = np.array([list(map(tweet_to_coords, neg_tweets))]).squeeze()

In [76]:
X = np.concatenate((pos_tweets_coords, neg_tweets_coords))

In [64]:
y = np.concatenate((np.ones(len(pos_tweets)), np.zeros(len(neg_tweets))))

In [77]:
random_idxs = np.random.permutation(len(y))
X = X[random_idxs]
y = y[random_idxs]

In [80]:
N_train = int(0.8*len(y))

X_train, X_test = X[:N_train], X[N_train:]
y_train, y_test = y[:N_train], y[N_train:]

In [87]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), linear_model.SGDClassifier(max_iter=1000, tol=1e-4))


clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier(tol=0.0001))])

In [88]:
(clf.predict(X_test) == y_test).mean()

0.6102